In [13]:
import pandas as pd
import numpy as np
import os
import re

In [14]:
file_path = os.listdir("rbp_samplealiquot")
r = re.compile(r".*_SampleAliquot_output\.csv")
newlist = list(filter(r.match, file_path))

In [15]:
october_rbp = []
for i in newlist:
    RBP_output = pd.read_csv(f'rbp_samplealiquot/{i}')
    if re.match(r'2020-10-\d{2}', RBP_output.iloc[0,1]):
        october_rbp.append([RBP_output.iloc[8,1], RBP_output.iloc[7,1], RBP_output.iloc[0,1]])

In [16]:
key = pd.DataFrame(october_rbp, columns = ['RBP', 'SRP', 'Date'])  
key

,RBP,SRP,Date
0,RBP200828A121,SRP200910A186,2020-10-23
1,RBP200828A122,SRP200910A188,2020-10-23
2,RBP200827A074,SRP200901A345,2020-10-19
3,RBP200828A311,SRP200910A121,2020-10-31
4,RBP200827A092,SRP200901A042,2020-10-13
...,...,...,...
266,RBP200828A072,SRP200910A175,2020-10-21
267,RBP200828A247,SRP200901A251,2020-10-08
268,RBP200828A135,SRP200910A112,2020-10-31
269,RBP200828A204,SRP200901A253,2020-10-07


In [17]:
path = os.listdir("/ghds/cv19/analysis")
r2 = re.compile("2010.{27}$")
october_runs = list(filter(r2.match, path))

In [18]:
october_df = []
for i in october_runs:
    file = pd.read_csv(f"/ghds/cv19/analysis/{i}/c19_call.hdr.tsv", sep='\t')
    file = file[file['sample_type'] == 'Sample']
    file.drop(file[file['pos_tube_rack'].str.match(r'.{14}(A6|B5)')==True].index, inplace=True)
    
    file['RBP_plate'] = [value[:13] for _,value in file['pos_tube_rack'].items()]
    october_df.append(file)
    
df = pd.concat(october_df)
df = df.merge(key, left_on='RBP_plate', right_on='RBP', how='outer')

In [19]:
negative = []
for i in october_rbp:
    x = df[df['RBP'] == i[0]]['call'].unique()
    if len(x) == 1 and x[0] == 'not_detected':
        negative.append(i)

negative.sort(key=lambda x:x[2], reverse=True)

In [27]:
oct_neg = [[df[df['RBP'] == i[0]]['call'].unique(), i[0], i[1], i[2], df[df['RBP'] == i[0]]['call'].count()] for i in negative]
negative_clinical_matrix = pd.DataFrame(oct_neg, columns=['Call', 'RBP', 'SRP', 'Date', 'Total Count'])
negative_clinical_matrix[negative_clinical_matrix['Total Count'] == 92]\
    .sort_values(by=['Date', 'SRP'], ascending=False)


,Call,RBP,SRP,Date,Total Count
1,[not_detected],RBP200828A307,SRP200910A113,2020-10-31,92
2,[not_detected],RBP200828A163,SRP200910A053,2020-10-31,92
5,[not_detected],RBP200828A141,SRP200910A105,2020-10-29,92
6,[not_detected],RBP200828A143,SRP200910A103,2020-10-29,92
7,[not_detected],RBP200828A142,SRP200910A054,2020-10-29,92
...,...,...,...,...,...
127,[not_detected],RBP200827A219,SRP200901A255,2020-10-02,92
126,[not_detected],RBP200827A199,SRP200901A254,2020-10-02,92
125,[not_detected],RBP200827A177,SRP200901A230,2020-10-02,92
124,[not_detected],RBP200827A232,SRP200901A229,2020-10-02,92
